In [ ]:
using Revise

In [ ]:
using MeshCatMechanisms
using MeshCat
using RigidBodyDynamics
using Gurobi
using Flux
using ProgressMeter
using MLDataPattern
using JLD2
using Plots; gr()

In [ ]:
import LCPSim
import LearningMPC
import Hoppers

In [ ]:
robot = Hoppers.Hopper()
mvis = MechanismVisualizer(robot.mechanism, URDFVisuals(Hoppers.urdf))
IJuliaCell(mvis)

In [ ]:
function create_net()
    net = Chain(
        Dense(4, 16, elu),
        Dense(16, 16, elu),
        Dense(16, 1)
    )
    loss = (x, lb, ub) -> begin
        y = model(x)
        sum(ifelse(y < lb, lb - y, ifelse(y > ub, y - ub, 0 * y)))
    end
    net, loss
end

In [ ]:
net, loss = create_net()
net_params = params(net)
optimizer = Flux.Optimise.ADADelta(net_params)

xstar = BoxValkyries.nominal_state(robot)

mpc_params = LearningMPC.MPCParams(
    Δt=0.05,
    horizon=10,
    mip_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0, 
        TimeLimit=5, 
        MIPGap=1e-1,
        FeasibilityTol=1e-3),
    lcp_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0))

Q, R = Hoppers.default_costs(robot)
foot = findbody(robot.mechanism, "foot")

B_matrices = map([Point3D(default_frame(foot), 0., 0., 0.)], Point3D[]) do contacts
    ustar = LearningMPC.nominal_input(xstar, contacts)
    

contacts = [Point3D(default_frame(foot), 0., 0., 0.)]

A, B, c = LCPSim.ContactLQR.contact_linearize(xstar, , Jc)
B_matrices = Matrix{Float64}()


In [ ]:
xstar = Hoppers.nominal_state(robot)

Q, R = Hoppers.default_costs(robot)
foot = findbody(robot.mechanism, "foot")
Δt = 0.05
contacts = [Point3D(default_frame(foot), 0., 0., 0.)]
ustar = LearningMPC.nominal_input(xstar, contacts)
Jc = LCPSim.ContactLQR.contact_jacobian(xstar, contacts)
@show Jc
A, B, c = LCPSim.ContactLQR.contact_linearize(xstar, ustar, Jc)
B

In [ ]:
xstar = Hoppers.nominal_state(robot)

Q, R = Hoppers.default_costs(robot)
foot = findbody(robot.mechanism, "foot")
Δt = 0.05
contacts = Point3D[]
ustar = LearningMPC.nominal_input(xstar, contacts)
@show ustar
Jc = LCPSim.ContactLQR.contact_jacobian(xstar, contacts)
@show Jc
A, B, c = LCPSim.ContactLQR.contact_linearize(xstar, ustar, Jc)
B

In [ ]:
net, loss = create_net()
net_params = params(net)
optimizer = Flux.Optimise.ADADelta(net_params)

xstar = BoxValkyries.nominal_state(robot)

mpc_params = LearningMPC.MPCParams(
    Δt=0.05,
    horizon=10,
    mip_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0, 
        TimeLimit=5, 
        MIPGap=1e-1, 
        MIPGapAbs=5e-1,
        FeasibilityTol=1e-3),
    lcp_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0))

Q, R = BoxValkyries.default_costs(robot)
feet = findbody.(robot.mechanism, ["lf", "rf"])
lqrsol = LearningMPC.LQRSolution(xstar, Q, R, mpc_params.Δt, Point3D.(default_frame.(feet), 0., 0., 0.))
lqrsol.S .= 1 ./ mpc_params.Δt .* Q


Jc = LCPSim.ContactLQR.contact_jacobian(xstar, [Point3D(default_frame(foot), 0., 0., 0.)])
A, B, c = LCPSim.ContactLQR.contact_linearize(xstar, ustar, Jc)

net_controller = x -> Flux.Tracker.data(net(state_vector(x)))

mpc_controller = LearningMPC.MPCController(robot.mechanism, 
    robot.environment, mpc_params, lqrsol, 
    [net_controller, lqrsol]);

sample_sink = LearningMPC.MPCSampleSink{Float64}(true)
playback_sink = LearningMPC.PlaybackSink(mvis, mpc_params.Δt)

mpc_controller.callback = LearningMPC.call_each(
    sample_sink,
    playback_sink,
)

live_viewer = LearningMPC.live_viewer(mvis)


dagger_controller = LearningMPC.call_each(
    LearningMPC.dagger_controller(
        mpc_controller,
        net_controller,
        0.2),
    live_viewer
    )

termination = x -> begin
    (configuration(x)[2] < 0.5 || 
     configuration(x)[3] > π/4 ||
     configuration(x)[3] < -π/4)
end

dataset = LearningMPC.Dataset(lqrsol)

function collect_into!(data::Vector{<:LearningMPC.Sample})
    empty!(sample_sink)
    LearningMPC.randomize!(x0, xstar, 0.0, 1.5)
    results = LCPSim.simulate(x0, 
        dagger_controller,
        robot.environment, mpc_params.Δt, 30, 
        mpc_params.lcp_solver;
        termination=termination);
    append!(data, sample_sink.samples)
end

x0 = MechanismState{Float64}(robot.mechanism)

features(s::LearningMPC.Sample) = (s.state, s.uJ[:, 1])

In [ ]:
datasets = Vector{LearningMPC.Dataset{Float64}}()
all_training_data = Vector{Tuple{Vector{Float64}, Vector{Float64}}}()
all_validation_data = Vector{Tuple{Vector{Float64}, Vector{Float64}}}()
losses = Vector{Tuple{Float64, Float64}}()

@showprogress for i in 1:20
    dataset = LearningMPC.Dataset(lqrsol)
    for i in 1:2
        collect_into!(dataset.training_data)
    end
    collect_into!(dataset.testing_data)
    collect_into!(dataset.validation_data)
    append!(all_training_data, features.(dataset.training_data))
    append!(all_validation_data, features.(dataset.validation_data))
    
    @time for i in 1:10
        Flux.train!(loss, shuffleobs(all_training_data), optimizer)
        push!(losses, 
            (mean(xy -> Flux.Tracker.data(loss(xy...)), 
                  all_training_data),
             mean(xy -> Flux.Tracker.data(loss(xy...)), 
                  all_validation_data)))
    end
    push!(datasets, dataset)
    
    jldopen("box-val-revolute.jld2", "w") do file
        file["datasets"] = datasets
        file["net"] = net
        file["lqrsol"] = lqrsol
        file["mpc_params"] = Dict(
            "Δt" => mpc_params.Δt,
            "horizon" => mpc_params.horizon,
        )
        file["all_training_data"] = all_training_data
        file["all_validation_data"] = all_validation_data
        file["losses"] = losses
    end
    
    plt = plot(first.(losses), label="training")
    plot!(plt, last.(losses), label="validation")
    ylims!(plt, (0, ylims(plt)[2]))
    display(plt)
end

In [ ]:
x0 = MechanismState{Float64}(robot.mechanism)
LearningMPC.randomize!(x0, xstar, 0.0, 1.0)
results = LCPSim.simulate(x0, 
    net_controller,
    robot.environment, mpc_params.Δt, 100, 
    mpc_params.lcp_solver,
    termination=x -> false);
LearningMPC.playback(mvis, results, 0.05)

In [ ]:
online_mpc_controller = LearningMPC.OnlineMPCController(
    robot.mechanism,
    robot.environment,
    mpc_params,
    lqrsol,
    [lqrsol, net_controller]
);

In [ ]:
x0 = MechanismState{Float64}(robot.mechanism)
LearningMPC.randomize!(x0, xstar, 0.0, 1.0)
results = LCPSim.simulate(x0, 
    online_mpc_controller,
    robot.environment, mpc_params.Δt, 100, 
    mpc_params.lcp_solver,
    termination=x -> false);
LearningMPC.playback(mvis, results, 0.05)

In [ ]:
LearningMPC.playback(mvis, results, 0.05)